In [1]:
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import re
from io import StringIO
import PyPDF2
import pandas as pd
from datetime import datetime 
import datetime as dt
from timeit import default_timer as timer
pd.set_option('display.max_rows', 500)

In [2]:
folder = "../Dec2020"
resultFolder = "Dec2020_Result"
file = "12.31 3207971832_2020-December_eStatement.pdf"
path = folder + '/' + file
excelFile = '../' + resultFolder + '/' + re.sub('\.', '_', '31_12 ' + file.split('-')[0].split(' ')[1]) + '.xlsx'
excelFile

'../Dec2020_Result/31_12 3207971832_2020.xlsx'

In [3]:
date = re.sub('\.', '/', file.split(' ')[0])
previousDate = (datetime.strptime(date, "%m/%d") - dt.timedelta(days=1)).strftime('%d/%m')
#for nov
date = datetime.strptime(date, "%m/%d").strftime('%d/%m')
date, previousDate

('31/12', '30/12')

In [4]:
def convertPdfToTxt(path):
    startTime = timer()
    rsrcmgr = PDFResourceManager()
    retstr = StringIO()
    #codec = 'utf-8'
    laparams = LAParams()
    device = TextConverter(rsrcmgr, retstr, laparams=laparams)
    fp = open(path, 'rb')
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    password = ""
    maxpages = 0
    caching = True
    pagenos=set()
    for page in PDFPage.get_pages(fp, pagenos, maxpages = maxpages,caching = caching, check_extractable = True):
        interpreter.process_page(page)
    text = retstr.getvalue()
    fp.close()
    device.close()
    retstr.close()
    endTime = timer()
    print("Total time: %0.4fs" % (endTime - startTime))
    return text

text = convertPdfToTxt(path)

#extract no pages
noPage = len(re.findall("NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", text))

noPage

Total time: 25.8695s


869

In [5]:
accountInfoDict = {}
accountCol = ["Nombor Akaun / Account Number", 'Jenis Akaun / Account Type',
             'Tarikh Penyata / Statement Date', 'RINGKASAN / SUMMARY']
for no, desc in enumerate(accountCol):
    if no != len(accountCol)-1:
        descIndex = text.index(desc)
        descLen = len(desc)
        nextDescIndex = text.index(accountCol[no + 1])
        result = text[descIndex + descLen: nextDescIndex]
        accountInfoDict[desc] = result
accountInfoDict

{'Nombor Akaun / Account Number': '3207971832',
 'Jenis Akaun / Account Type': 'RM Plus Current Account',
 'Tarikh Penyata / Statement Date': '31 Dec 2020'}

In [6]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

def extractAmount(desc, greedy = True):
#     greedyPattern = '([\d\.\,]+)'
    greedyPattern = '([\d\.\,]+\.\d{2})'
    notGreedyPattern = '([\d\.\,]+?\.\d{2}?)'
    try:
        if greedy:
            result = re.search(greedyPattern, desc).group(1)
            return result
        else:
            result = re.search(notGreedyPattern, desc).group(1)
            return result
    except Exception as e:
#         print(e)
        print("Wrong number pattern")

In [57]:
summaryDict = {}
summaryCol = ['Baki Penutup / Closing Balance',
              'Jumlah Debit / Total Debits', 'Bil. Debit / No. of Debits',
              'Jumlah Kredit / Total Credits', 'Bil. Kredit / No. of Credits']
#extract series of number 
numberSeries = extractText(text, summaryCol[4], 'Muka Surat 1')[0]

#extract closing balance amount
closingBalanceAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[0]] = closingBalanceAmount
numberSeries = re.sub(closingBalanceAmount, '', numberSeries)

#extract debit amount
debitAmount = extractAmount(numberSeries, greedy = False)
summaryDict[summaryCol[1]] = debitAmount
numberSeries = re.sub(debitAmount, '', numberSeries)

#credit no credit
noCredit = re.sub(extractAmount(numberSeries, greedy = False), '', numberSeries)
summaryDict[summaryCol[4]] = noCredit
numberSeries = re.sub(noCredit + '$', '', numberSeries)

tempNumberSeries = numberSeries
creditAmount3 = re.search('(\d{3}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit3 = re.sub(creditAmount3, '', tempNumberSeries)
creditAmount1 = re.search('(\d{2}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit1 = re.sub(creditAmount1, '', tempNumberSeries)
creditAmount2 = re.search('(\d{1}\,(\d+\,)*\d+\.\d{2})', numberSeries).group(1)
noDebit2 = re.sub(creditAmount2, '', tempNumberSeries)

noDebitList = [noDebit1, noDebit2, noDebit3]
creditAmountList = [creditAmount1, creditAmount2, creditAmount3]
noDebitList, creditAmountList

(['95926', '959263', '9592'], ['39,091.17', '9,091.17', '639,091.17'])

In [9]:
def returnTableList(no):
    tempList = []
    textCopy = text
    for i in range(1, no + 1):
#     for i in range(1, 3):
        table, remain = extractText(textCopy, 'Page %s of %s'%(i, no), 'Penyata ini dicetak melalui komputer.')
        index = remain.index('Penyata ini dicetak melalui komputer.')
        termLen = len('Penyata ini dicetak melalui komputer.')
        textCopy = remain[index + termLen:]
        table = re.sub("TARIKHURUS NIAGADEBITKREDITBAKIDATETRANSACTIONDEBITCREDITBALANCE", '', table)
        tempList.append(table)
    return tempList

tableList = returnTableList(noPage)

len(tableList)

869

In [43]:
#extract first line of transaction

#trx date, cheque no., ref1, ref2, ref3, ref4, credit amount, debit amount, balance
resultList = []

#extract content of first table from text
table = tableList[0]

#extract balance amount
balanceAmount, tempText = extractText(table, "Balance From Last Statement", date)

#store record of balance from last statement
# resultList.append([previousDate, '','', '','', "Balance From Last Statement", '', '', balanceAmount])
#trx date, ref1, credit amount, debit amount, balance
resultList.append([previousDate, "Balance From Last Statement", '', '', balanceAmount])

In [44]:
def extractText(desc, startStr, endStr):
    try:
        startStrIndex = desc.index(startStr)
        startStrLen = len(startStr)
        endStrIndex = desc.index(endStr)         
        if startStr == '':
            result = desc[:endStrIndex]
            return result
        elif endStr!='':
            result = desc[startStrIndex + startStrLen:endStrIndex]
            remainDesc = desc[endStrIndex:]
            return result, remainDesc
        else:
            result = desc[startStrIndex + startStrLen:]
            return result
        
    except Exception as e:
#         print(e)
        print("Failed to extract text")

In [45]:
#extract amountList and descList
#steps: 
#1. extract first and second number, store first number in descList
#2. extract desc between first and second number
#3. store desc in descList
#repeat 1 till 3 until closing balance's line

amountList = []
descList = []
closingLine = 0

# for no, table in enumerate([tableList[0]]):
for no, table in enumerate(tableList):
    if no == 0:
        tempText = tempText
    else:
        tempText = table
    # for i in range(3):
    while closingLine == 0:
        #remove date first
        if re.search(date, tempText):
            tempText = re.sub(date, '', tempText)
        #remove RMB @1.55
        tempText = re.sub("RMB.*@ \d\.\d+", '', tempText)
        if no in [469, 839]:    
        #remove 9.10726135944E11: table 469, 839
            tempText = re.sub("9.10726135944E11", '', tempText)
        #extract amount first
        tempAmount1 = extractAmount(tempText)
        #store amount
        amountList.append(tempAmount1)
        #temporary subbed text to get second amount
        tempSubText = re.sub(tempAmount1, '', tempText)
        tempAmount2 = extractAmount(tempSubText)
        tempDesc = extractText(tempSubText, '', tempAmount2)
        
        #if Balance C/F, last line of transaction for that page
        if re.search(r"Balance C/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance C/F", "", str(tempDesc))
            amountList.append(tempAmount2)
            descList.append(tempDesc)
            descList.append("Balance C/F")
            break
            
        #if Balance B/F, first line of transaction for that page
        elif re.search(r"Balance B/F", str(tempDesc)):
            tempDesc = re.sub(r"Balance B/F", "", str(tempDesc))
            descList.append("Balance B/F")
            tempAmount1 = extractAmount(re.sub(date, '', tempText), greedy = False)
            amountList[-1] = tempAmount1
            tempAmount1Index = tempText.index(tempAmount1)
            tempAmount1Len = len(tempAmount1)
            tempText = tempText[tempAmount1Index+tempAmount1Len:]
            
        #if last line for whole file
        elif re.search("Closing Balance In This Statement", str(tempDesc)):
            closingAmount = tempAmount2
            closingAmountIndex = tempSubText.index(closingAmount)
            lastDesc = tempSubText[:closingAmountIndex]
            #sub closing amount with '' in lastDesc
            lastDesc = re.sub("Closing Balance In This Statement", '', lastDesc)
            descList.append(lastDesc)

            #store last line - amount and desc
            amountList.append(closingAmount)
            descList.append("Closing Balance In This Statement")
#             closingLine = 1
            break
        #extracted all ady
        elif not tempAmount2:
#             closingLine = 1
            break
        else:
            tempText = re.sub('^' + re.escape(tempDesc), '', tempSubText)
            descList.append(tempDesc)

In [32]:
tableList[839]

"31/12Balance B/F4,297,650.32FEE 3207971832PM31122099EU REFUND EU REFUND 23122020LAU TING SUNG0.104,297,650.22HANDLING CHRG 005387FEE 3207971832PM31122099EU REFUND EU REFUND 23122020DING CHEE LING0.104,297,650.12HANDLING CHRG 005388FEE 3207971832PM31122099EU REFUND EU REFUND 23122020LAM HUNG MING0.104,297,650.02HANDLING CHRG 005389FEE 3207971832PM31122099EU REFUND EU REFUND 23122020TING LEONG TECK0.104,297,649.92HANDLING CHRG 005390FEE 3207971832PM31122099EU REFUND EU REFUND 23122020PHAN GUANG NING0.104,297,649.82HANDLING CHRG 005391FEE 3207971832PM31122099EU REFUND EU REFUND 23122020T'NG KAR POH0.104,297,649.72HANDLING CHRG 005392FEE 3207971832PM31122099EU REFUND EU REFUND 23122020KIING TIN TIN0.104,297,649.62HANDLING CHRG 005393FEE 3207971832PM31122099EU REFUND EU REFUND 23122020JOYCE LING TSUI0.104,297,649.52HANDLING CHRG 005394FEE 3207971832PM31122099EU REFUND EU REFUND 23122020LIM NYET YUN0.104,297,649.42HANDLING CHRG 005395FEE 3207971832PM31122099EU REFUND EU REFUND 231220209.107

In [33]:
tableList[469]

'31/12Balance B/F4,558,650.72MERCHANT REDEEM MERCHANT REDDEEM28122020 TEN LEE KHIAW0.104,558,650.62HANDLING CHRG 003315FEE 3207971832PM31122098COP REFUND COP REFUND 28122020LIM WEI CHUN1,264.004,557,386.62DR-ECP 0033153207971832PM31122099 EU REFUNDEU REFUND 23122020 9.10726135944E110.104,557,386.52HANDLING CHRG 003316FEE 3207971832PM31122001MERCHANT REDEEM MERCHANT REDDEEM28122020 CHUA SIAM KENG0.104,557,386.42HANDLING CHRG 003316FEE 3207971832PM31122098COP REFUND COP REFUND 28122020YEE JIN CHU0.104,557,386.32HANDLING CHRG 003317FEE 3207971832PM31122001MERCHANT REDEEM MERCHANT REDDEEM28122020 EKA RESTAURANT S0.104,557,386.22HANDLING CHRG 003317FEE 3207971832PM31122098COP REFUND COP REFUND 28122020TIONG KIEW HEE997.004,556,389.22DR-ECP 0033173207971832PM31122099 EU REFUNDEU REFUND 23122020 DANIEL WONG CHEN0.104,556,389.12HANDLING CHRG 003318FEE 3207971832PM31122001MERCHANT REDEEM MERCHANT REDDEEM28122020 SNOOZE COMFORT B0.104,556,389.02HANDLING CHRG 003318FEE 3207971832PM31122098COP REF

In [46]:
def splitAmount(desc):
    try:
        #remove first . before num if exists
        desc = re.sub('^\.', '', desc)
        #remove date pattern
        desc = re.sub('^\d{1,2}\.\d{1,2}\.\d{4}', '', desc)
        amountPattern = '(.*?\.\d{2}?)'
        result = re.search(amountPattern, desc).group(1)
        desc2 = re.sub('^' + result, '', desc)
        return result, desc2
    except Exception as e:
        print(e)
        print("Wrong amount pattern")

splitAmountList = [splitAmount(i) for i in amountList]

In [47]:
def checkCreditDebit(amountTuple, previousBalance):
    #exception:
    amount1 = float(re.sub(',', '', amountTuple[0]))
#     currentBalance = re.sub('^\.', '', amountTuple[1])
    currentBalance = amountTuple[1]
#     print(currentBalance)
    if currentBalance !='':
        currentBalance = float(re.sub(',', '', currentBalance))
        previousBalance = float(re.sub(',', '', previousBalance))
        if currentBalance - previousBalance == amount1:
            amountTuple = ('', round(amount1, 2), currentBalance)
        elif previousBalance - currentBalance == amount1:
            amountTuple = (round(amount1, 2), '', currentBalance)
        #if amount 1 is wrong
        elif currentBalance > previousBalance:
            credit  = currentBalance - previousBalance
            amountTuple = ('', round(credit,2), currentBalance)
        elif previousBalance > currentBalance:
            debit  = previousBalance - currentBalance
            amountTuple = (round(debit,2), '', currentBalance)            
    else:
        amountTuple = ('', '', amount1)
    return amountTuple, str(amountTuple[2])

#regex pattern based on transDesc
transTypePatternDict = {'INSTANT TRSF CR':'(.*CR)\s*(\d{6})(.+)',
                       'TSFR FUND DR-ATM/EFT':'(TSFR FUND DR-ATM/EFT)\s*(\d{6})(.+)'}

def splitStr(desc):
    foundPattern = 0
    for transType in transTypePatternDict:
        if re.search(transTypePatternDict[transType], desc):
            pattern = transTypePatternDict[transType]
            transType = re.search(pattern, desc).group(1)
            chequeNo = re.search(pattern, desc).group(2)
            ref1  = re.search(pattern, desc).group(3)
            foundPattern = 1
            finalResult = [chequeNo, transType, ref1] + ['']*2
            break
    if foundPattern == 0:
        finalResult = ['']*2 + [desc] + ['']*2
    #[chequeNo, transactionDescType, ref1, ref2, ref3]
    return finalResult

In [48]:
#categorize one amount as credit/debit
splitAmountList2 = []
for no, i in enumerate(splitAmountList):
    if no == 0:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, balanceAmount)
        splitAmountList2.append(resultTuple)
    else:
        resultTuple, previousBalanceAmount = checkCreditDebit(i, previousBalanceAmount)
        splitAmountList2.append(resultTuple)

# splitStrList = [splitStr(i) for i in descList]

# #column names
# colList = ['Trn. Date', 'Cheque No/Ref No', 'Transaction Description',
#            'Debit Amount', 'Credit Amount', 
#            'Reference 1', 'Reference 2', 'Reference 3', 'Balance']

# for no, finalStrList in enumerate(splitStrList):
#     tempList =  [date] + [i for i in finalStrList[:2]]
#     tempList += [i for i in splitAmountList2[no][:2]] + [i for i in finalStrList[2:]] + [splitAmountList2[no][-1]]
#     resultList.append(tempList)
    
colList = ['Trn. Date','Reference 1',
           'Debit Amount', 'Credit Amount', 'Balance']
for no, finalStrList in enumerate(descList):
    tempList =  [date] + [finalStrList]
    tempList +=  [i for i in splitAmountList2[no]]
    resultList.append(tempList)

In [49]:
df = pd.DataFrame(resultList, columns = colList)
df

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
0,30/12,Balance From Last Statement,,,"28,363,546.76"
1,31/12,SI CR FROM 3999XXXXXX,,46554.1,2.84101e+07
2,31/12,SI CR FROM 3999XXXXXX,,339602,2.87497e+07
3,31/12,RTN ITEM 091847IBG RTN ITEM 20201231 LTCS28201...,,1000,2.87507e+07
4,31/12,RTN ITEM 091849IBG RTN ITEM 20201231 LTCS28201...,,1769,2.87525e+07
...,...,...,...,...,...
11403,31/12,GIRO PYMT-ATM/EFT 625442INDAH WATER KONSORTIUM,40,,3.58142e+06
11404,31/12,INSTANT TRSF DR 871910NBS INTELLECTUAL SDN PI-...,2320,,3.5791e+06
11405,31/12,GIRO PYMT-ATM/EFT 872277JOMPAY - 7CW48XYW - E-...,57584.5,,3.52152e+06
11406,31/12,INSTANT TRSF DR 872307SHAHRIZAL BIN YUSRI RIDE...,440,,3.52108e+06


In [25]:
max([i for i in df['Credit Amount'] if type(i)!=str])

4557377.42

In [26]:
df.query("`Credit Amount`==4557377.42")

,Trn. Date,Reference 1,Debit Amount,Credit Amount,Balance
6497,31/12,HANDLING CHRG 003316FEE 3207971832PM31122001ME...,,4.55738e+06,4.55739e+06


In [27]:
df.query("`Credit Amount`==4557377.42")['Reference 1'].values

array(['HANDLING CHRG 003316FEE 3207971832PM31122001MERCHANT REDEEM MERCHANT REDDEEM28122020 CHUA SIAM KENG'],
      dtype=object)

In [50]:
df.loc[df.shape[0]-1]['Balance']

3521077.94

In [51]:
sum([i for i in df['Debit Amount'] if i!=''])

25481559.99000702

In [52]:
trueNoDebit = df.query("`Debit Amount`!=''").shape[0]
trueNoDebit

9592

In [53]:
sum([i for i in df['Credit Amount'] if i!=''])

639091.1699999999

In [58]:
#confirm no debit
noDebitIndex = noDebitList.index(str(trueNoDebit))
creditAmount = creditAmountList[noDebitIndex]
summaryDict[summaryCol[2]] = trueNoDebit
summaryDict[summaryCol[3]] = creditAmount

summaryDict

{'Baki Penutup / Closing Balance': '3,521,077.94',
 'Jumlah Debit / Total Debits': '25,481,559.99',
 'Bil. Kredit / No. of Credits': '78',
 'Bil. Debit / No. of Debits': 9592,
 'Jumlah Kredit / Total Credits': '639,091.17'}

In [59]:
excelFile

'../Dec2020_Result/31_12 3207971832_2020.xlsx'

In [60]:
import xlsxwriter 
  

workbook = xlsxwriter.Workbook(excelFile)
worksheet = workbook.add_worksheet() 

# Rows and columns are zero indexed. 
row = 0

for col in accountInfoDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, accountInfoDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
for col in summaryDict : 
  
    # write operation perform 
    worksheet.write(row, 0, col + ':')
    worksheet.write(row, 1, summaryDict[col])
  
    # incrementing the value of row by one 
    # with each iteratons. 
    row += 1
    
workbook.close()

from openpyxl import load_workbook
#write pandas 
writer = pd.ExcelWriter(excelFile, engine='openpyxl')
writer.book = load_workbook(excelFile)
# copy existing sheets
writer.sheets = dict((ws.title, ws) for ws in writer.book.worksheets)
reader = pd.read_excel(excelFile)
df.to_excel(writer, sheet_name='Sheet1', startrow = len(reader) + 2, header = True, index = False)
writer.save()